# Workshop 03: Leonardo's centralized church

## Table of contents
1. [Modelli originali](#p1)
2. [Codice](#p2)
    1. [Funzioni e variabili di supporto](#sp1)
    2. [Guglia principale](#sp2)
    3. [Guglia secondaria](#sp3)
    4. [Edificio principale](#sp4)
    5. [Strutture esterne](#sp5)
    6. [Chiesa completa](#sp6)


## Church's original model <a name="p1"></a>
![img1](imgs/church.jpg)
![img2](imgs/nuvoli.jpg)

## Codice <a name="p2"></a>

In [1]:
from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 0.005513 seconds


### Funzioni e variabili di supporto <a name="sp1"></a>
 - I colori utilizzati per i vari componenti della struttura: beige, carminio ed avorio.
 - halfSphere è la funzione utilizzata per generare metà di una sfera.
 - arc genera un arco dati raggio, larghezza e spessore.
 - arcs genera 2 coppie di archi, a distanza angolare di PI/2; i due archi appartenenti alla coppia sono posti ad una distanza angolare data dal parametro in input.
 - grassFloor genera un terreno d'erba; le dimensioni sono date in input

In [2]:
BEIGE = Color4f([.77, .72, .51, 1.])
IVORY = Color4f([1., 1., .94, 1.])
CARMINE = Color4f([.59, 0, 0.1, 1.])

def halfSphere (radius):
    '''
    generates the model of the upper half of a sphere
    @param: radius
    '''
    def GSPHERE0 (subds):
        N , M = subds
        domain = Hpc(Grid([N*[PI/N],M*[PI/M]]))
        domain = MAT([[1,0,0,0],[-PI*2,1,0,0],[-PI,0,1,0],[0,0,0,1]])(domain)
        fx  = lambda p: radius * math.cos(p[0]) * math.sin  (p[1])
        fy  = lambda p: radius * math.cos(p[0]) * math.cos (p[1])
        fz  = lambda p: radius * math.sin(p[0])
        ret=  GMAP([fx, fy, fz])(domain)
        return R([1,2])(PI)(ret)
    return GSPHERE0

def arc(radius, width, border):
    '''
    generates model of an arc of given radius, width and border
    @param: radius
    @param: width
    @param: border
    '''
    cylinder = CYLINDER([radius+border,width])(200)
    voidCylinder = CYLINDER([radius,width])(200)
    cuboidY = radius+border
    cuboidX = 2 * cuboidY
    voidCuboid = T([1])([-cuboidX / 2])(CUBOID([cuboidX, cuboidY, width]))
    arc = COLOR(BEIGE)(STRUCT([DIFF([cylinder, voidCylinder, voidCuboid])]))
    return R([2,3])(-PI / 2)(arc)
    
def arcs(r, w, b, angle):
    '''
    generates a composition of arcs
    @param: radius
    @param: width
    @param: border
    @param: angle between arc couple
    '''
    a = arc(r, w, b)
    arcCouple = STRUCT([a, R([1,2])(angle)(a)])
    return STRUCT([arcCouple, R([1,2])(PI / 2)(arcCouple)])

def grassFloor(x, y):
    '''
    generates model for the floor using grass texture
    @param: width
    @param: depth
    '''
    texture = TEXTURE(["imgs/GrassGreenTexture0003.jpg", TRUE, TRUE, 0, 0, 0, 5.0, 5.0])
    return texture(CUBOID([x, y]))

VIEW(arc(5, .5, .2))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f25651004b0> >

![arc](imgs/arc.png)

### Funzione per la generazione della guglia principale <a name="sp2"></a>
 - La guglia principale è la più grande della struttura ed è posta centralmente sopra l'edificio principale.
 - È caratterizzata da una punta più elaborata ed una disposizione degli archi leggermente diversa (gli archi all'interno di una coppia sono più vicini rispetto alla distanza tra le due coppie); anche lo strato tra il pilastro e la cupola è diverso da quello delle altre guglie.

In [6]:
def mainDome(radius):
    '''
    generates the model for the main dome of the church
    @param: radius
    '''
        
    def point(r, h):
        hFraction = .05 * h;
        base = CYLINDER([1.8 * r, 2 * hFraction])(30)
        lowerBlock = TOP([base, CUBOID([1.8 * r, 1.8 * r, hFraction])])
        middleBlock = TOP([lowerBlock, CUBOID([1.5 * r, 1.5 * r, 6. * hFraction])])
        block = TOP([middleBlock, CUBOID([1.8 * r, 1.8 * r, hFraction])])
        base2 = CYLINDER([.7 * r, 2. * hFraction])(30)
        top = TOP([base2, CONE([.7 * r, 7. * hFraction])(30)])
        return TOP([block, top])
        
    sphereLevel = .94 * radius
    layer1 = COLOR(BEIGE)(CYLINDER([1.05 * radius, .1 * radius])(100))
    layer2 = COLOR(IVORY)(CYLINDER([1.03 * radius, .04 * radius])(100))
    
    #main components
    base = COLOR(BEIGE)(CYLINDER([1.05 * radius, .2 * radius])(100))
    sphere = COLOR(CARMINE)(JOIN(halfSphere(radius)([30,30])))
    pillar = COLOR(IVORY)(CYLINDER([radius, radius / 2])(100))
    middleLayer = TOP([ TOP([layer1, layer2]), layer1 ])
    point = COLOR(BEIGE)(point(.1 * radius, .6 * radius))
    arcsComposition = arcs(radius, .1 * radius, .05 * radius, PI / 5)
    
    #composition
    dome = TOP([base, pillar])
    dome = TOP([dome, middleLayer])
    dome = TOP([dome, sphere])
    dome = STRUCT([dome, T([3])([sphereLevel])(arcsComposition), T([3])([sphereLevel+radius])(point)])
    return dome


VIEW(mainDome(10))

Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f2565100f60> >

![dome](imgs/dome.png)

### Funzione per la generazione delle guglie secondarie <a name="sp3"></a>
 - Questa funzione viene utilizzata sia per generare le guglie più piccole nella parte superiore dell'edificio, sia per generare i pilastri con cupola delle strutture limitrofe all'edificio principale.
 - La punta non è elaborata come quella della guglia principale; gli archi sono disposti tutti con uguale distanza angolare.

In [41]:
def smallDome(radius, height):
    '''
    generates the model for the smaller domes
    @param: radius
    @param: height of the pillar element
    '''
    
    def point(r, h):
        return CONE([r, h])(30)
    
    arcsZ = 1.45 * radius
    pointZ = arcsZ + 1.05 * radius
    layer1 = CYLINDER([1.07 * radius, .05 * radius])(100)
    layer2 = CYLINDER([1.03 * radius, .1 * radius])(100)
    sphere = COLOR(CARMINE)(JOIN(halfSphere(radius)([30,30])))
    pointZT = 1.35 * radius + height
    
    #main components
    base = COLOR(BEIGE)(CYLINDER([1.05 * radius, .2 * radius])(100))
    pillar = COLOR(IVORY)(CYLINDER([radius, height])(100))
    middleLayer = COLOR(BEIGE)(TOP([layer2, layer1]))
    arcsComposition = arcs(radius, .1 * radius, .05 * radius, PI / 4)
    sphereArcs = ALIGN([[1, MED, MED], [2, MED, MED]])([sphere, arcsComposition])
    point = COLOR(BEIGE)(point(.1 * radius, .2 * radius))
    
    #composition
    dome = TOP([base, pillar])
    dome = TOP([dome, middleLayer])
    dome = TOP([dome, sphereArcs])
    dome = STRUCT([dome, T([1,2,3])([-.07 * radius, .03 * radius, pointZT])(point)])
    return dome
    
VIEW(smallDome(4,4))
#VIEW(smallDome(3,10))

Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f25646a6600> >

![small_dome](imgs/smallDome.png)
![pillar](imgs/pillar.png)

### Funzione per la generazione dell'edificio principale <a name="sp4"></a>
Genera l'edificio principale, ma viene usata anche per generare il blocco principale delle strutture esterne.

In [10]:
def mainBuilding(x, y, z):
    '''
    generates the model for the main building
    @param: width
    @param: depth
    @param: height
    '''
    
    lowerLevel = COLOR(BEIGE)(CUBOID([.97 * x, .97 * y, .13 * z]))
    middleLevel = COLOR(IVORY)(CUBOID([.94 * x, .94 * y, .8 * z]))
    upperLevel = COLOR(BEIGE)(CUBOID([x, y, .07 * z]))
    
    return TOP([ TOP([lowerLevel, middleLevel]) , upperLevel])

VIEW(mainBuilding(30, 30, 20))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f2565100c00> >

### Funzione per la generazione delle strutture esterne <a name="sp5"></a>
 Processo di generazione:
 
 - Attraverso la funzione mainBuilding viene generato il blocco principale.
 - Sopra il blocco principale viene posizionata una guglia.
 - Su 3 dei lati del blocco vengono posizionati dei pilastri con guglia.

In [52]:
def externalStructure(x, y, z):
    '''
    generates the model for the structures surrounding the church externally
    @param: width
    @param: depth
    @param: height
    '''
    
    dx = x / 8.
    dy = y / 8.
    dz = z / 7.
    domeZ = 2.5 * dz - 2.35 * dx
    pillarRadius = 1.8 * dx
    baseH = (.13 * 4.5 * dz) - (.2 * (pillarRadius))
    pillarBase = COLOR(BEIGE)(CYLINDER([1.05 * (pillarRadius), baseH])(100))
    pillarDome = smallDome(pillarRadius, 1.5 * dz)
    domeBase = COLOR(BEIGE)(CUBOID([4.5 * dx, 4.5 * dy, .2 * dz]))
    centralDome = smallDome(1.3 * dz, domeZ)
    
    #main components
    centralBlock = mainBuilding(6 * dx , 6 * dy, 4 * dz)
    dome = TOP([domeBase, centralDome])
    pillar = TOP([ pillarBase, pillarDome ])
    
    #composition
    structure = TOP([centralBlock, dome])
    structure = STRUCT([structure,\
                        T([1])([3 * dx])(pillar),\
                        T([1,2])([3 * dx, 5.65 * dy])(pillar),\
                        T([1,2])([5.8 * dx, 3 * dy])(pillar)\
                       ])
    return structure

VIEW(externalStructure(10, 10, 15))

Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f2564618840> >

![external](imgs/extStr.png)

### Funzione per la generazione della chiesa <a name="sp6"></a>

In [54]:
def church(x, y, z):
    '''
    generates the model of Leonardo's centralized church
    @param: width
    @param: depth
    @param: height
    '''
    #ratios
    ratioX = x / 80.
    ratioY = y / 80.
    ratioZ = z / 40.
    #main components
    bigDome = mainDome(10)
    building = mainBuilding(40, 40, 20)
    dome = smallDome(4.5, 3.5)
    extStr = externalStructure(20, 20, 26)
    floor = grassFloor(100, 100)
    
    #composition
    church = TOP([building, bigDome])
    church = STRUCT([church, \
                     T([1,2,3])([7, 7, 20])(dome),\
                     T([1,2,3])([7, 32, 20])(dome),\
                     T([1,2,3])([32, 7, 20])(dome),\
                     T([1,2,3])([32, 32, 20])(dome),\
                    ])
    church = STRUCT([church, \
                     T([1,2])([31, 12])(extStr),\
                     R([1,2])(PI / 2)(T([1,2])([31, -27])(extStr)),\
                     R([1,2])(PI)(T([1,2])([-8, -27])(extStr)),\
                     R([1,2])(3 * PI / 2), T([1,2])([-8, 12])(extStr),\
                    ])
    church = TOP([floor, church])
    return S([1,2,3])([ratioX, ratioY, ratioZ])(church)

VIEW(church(80, 80, 40))

Child 3
Child 3
Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f2564618a50> >

![church](imgs/churchFull.png)